In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("../data/datasets/train.csv")
test = pd.read_csv("../data/datasets/test.csv")

/home/gbfelgas/miniconda3/envs/cpe721/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
data.head()

,sessionId,fullVisitorId,visitStartTime,date,deviceCategory,operatingSystem,browser,country,city,trafficSource,...,productPagesViewed,addedToCart,isMobile,hour,day,month,isWeekend,isApple,lastVisitTime,hasRecentVisit
0,05682140582413690201471464743,568214058241369020,1471464743,2016-08-17 00:00:00+00:00,mobile,Windows Phone,Safari,Turkey,Izmir,google,...,0,0,1,-0.866025,-0.207912,-0.755750,0,0,0,0
1,13612048087514369511484615373,1361204808751436951,1484615373,2017-01-16 00:00:00+00:00,mobile,Windows Phone,Safari,United Kingdom,London,(direct),...,0,0,1,0.258819,-0.207912,0.000000,0,0,0,0
2,18823174067382551941474847129,1882317406738255194,1474847129,2016-09-25 00:00:00+00:00,mobile,Firefox OS,Firefox,Peru,La Victoria,youtube.com,...,0,0,1,-0.258819,-0.951057,-0.989821,1,0,0,0
3,18823174067382551941475342040,1882317406738255194,1475342040,2016-10-01 00:00:00+00:00,mobile,Firefox OS,Firefox,Peru,La Victoria,youtube.com,...,0,0,1,-0.965926,0.000000,-0.909632,1,0,1474847129,1
4,19074591080696861771476097343,1907459108069686177,1476097343,2016-10-10 00:00:00+00:00,mobile,Windows Phone,Safari,Vietnam,Ho Chi Minh City,youtube.com,...,0,0,1,0.258819,0.951057,-0.909632,0,0,0,0


In [4]:
data.columns

Index(['sessionId', 'fullVisitorId', 'visitStartTime', 'date',
       'deviceCategory', 'operatingSystem', 'browser', 'country', 'city',
       'trafficSource', 'trafficMedium', 'trafficCampaign', 'isFirstVisit',
       'totalVisits', 'totalHits', 'totalPageviews', 'totalTimeOnSite',
       'productPagesViewed', 'addedToCart', 'isMobile', 'hour', 'day', 'month',
       'isWeekend', 'isApple', 'lastVisitTime', 'hasRecentVisit'],
      dtype='object')

In [5]:
selected_data = data[[
    'deviceCategory',
    'operatingSystem',
    'browser',
    'country',
    'city',
    'isFirstVisit',
    'totalVisits',
    'totalHits',
    'totalPageviews',
    'totalTimeOnSite',
    'productPagesViewed',
    'isMobile',
    'hour',
    'day',
    'month',
    'isWeekend',
    'isApple',
    'hasRecentVisit',
    'addedToCart'
]]
selected_data.head()

,deviceCategory,operatingSystem,browser,country,city,isFirstVisit,totalVisits,totalHits,totalPageviews,totalTimeOnSite,productPagesViewed,isMobile,hour,day,month,isWeekend,isApple,hasRecentVisit,addedToCart
0,mobile,Windows Phone,Safari,Turkey,Izmir,1,1,1,1,0,0,1,-0.866025,-0.207912,-0.755750,0,0,0,0
1,mobile,Windows Phone,Safari,United Kingdom,London,1,1,1,1,0,0,1,0.258819,-0.207912,0.000000,0,0,0,0
2,mobile,Firefox OS,Firefox,Peru,La Victoria,1,1,4,4,95,0,1,-0.258819,-0.951057,-0.989821,1,0,0,0
3,mobile,Firefox OS,Firefox,Peru,La Victoria,0,1,2,2,61,0,1,-0.965926,0.000000,-0.909632,1,0,1,0
4,mobile,Windows Phone,Safari,Vietnam,Ho Chi Minh City,1,1,1,1,0,0,1,0.258819,0.951057,-0.909632,0,0,0,0


In [6]:
selected_data = selected_data[selected_data.totalTimeOnSite <= selected_data.query('totalTimeOnSite > 0')['totalTimeOnSite'].quantile(q=.99)]
selected_data = selected_data[selected_data.totalPageviews <= selected_data.query('totalPageviews > 0')['totalPageviews'].quantile(q=.99)]
selected_data = selected_data[selected_data.productPagesViewed <= selected_data.query('productPagesViewed > 0')['productPagesViewed'].quantile(q=.99)]

for column in ['totalTimeOnSite', 'totalPageviews', 'productPagesViewed']:
    selected_data.loc[ selected_data[selected_data[column] == 0].index, column ] = selected_data.query(column + ' > 0')[column].mean()

In [7]:
selected_data.head()

,deviceCategory,operatingSystem,browser,country,city,isFirstVisit,totalVisits,totalHits,totalPageviews,totalTimeOnSite,productPagesViewed,isMobile,hour,day,month,isWeekend,isApple,hasRecentVisit,addedToCart
0,mobile,Windows Phone,Safari,Turkey,Izmir,1,1,1,1.0,208.918857,2.218911,1,-0.866025,-0.207912,-0.755750,0,0,0,0
1,mobile,Windows Phone,Safari,United Kingdom,London,1,1,1,1.0,208.918857,2.218911,1,0.258819,-0.207912,0.000000,0,0,0,0
2,mobile,Firefox OS,Firefox,Peru,La Victoria,1,1,4,4.0,95.000000,2.218911,1,-0.258819,-0.951057,-0.989821,1,0,0,0
3,mobile,Firefox OS,Firefox,Peru,La Victoria,0,1,2,2.0,61.000000,2.218911,1,-0.965926,0.000000,-0.909632,1,0,1,0
4,mobile,Windows Phone,Safari,Vietnam,Ho Chi Minh City,1,1,1,1.0,208.918857,2.218911,1,0.258819,0.951057,-0.909632,0,0,0,0


In [8]:
selected_data['totalPageviews'] = np.log(1 + selected_data['totalPageviews'])

## Preprocessing

### Categorical
- `deviceCategory`: oneHotEncoding
- `isMobile`: cast to integer
- `browser`: ignored (device category is enough)
- `country`: get_continent_code and oneHotEncoding
- `city`: ignored
- `trafficSource`: ignored (?)
- `trafficMedium`: ignored (?)
- `trafficCampaign`: ignored (?)
- `operatingSystem`: binarize_os_category

### Numerical
- `isFirstVisit`: leave as is
- `totalVisits`: ignored
- `totalPageviews`: log (but we could binarize as well (like light/medium/heavy user)
* - `totalTimeOnSite`: substituir nulos por mediana dos não-nulos # threshold on 5
- `productPagesViewed`: binarize_product_pages_viewed
- `addedToCart`
- `hour`: get_cyclic_hour
- `day`: get_cyclic_day
- `month`: get_cyclic_month
- `isWeekend`: leave as is

In [9]:
def get_class_int(data: pd.DataFrame, col: list):
    int_data = data.copy()
    for c in col:
        cmap = {k: v for v, k in enumerate(data[c].unique())}
        cmap['other'] = len(cmap)
        int_data[c] = data[c].apply(lambda x: cmap[x])
    
    return int_data

In [11]:
x = get_class_int(selected_data, ['deviceCategory', 'operatingSystem', 'browser', 'country', 'city'])
y = x['addedToCart']
x.drop(columns=['addedToCart'], inplace=True)

In [14]:
x.to_csv("../data/datasets/x.csv", index=False)
y.to_csv("../data/datasets/y.csv", index=False)